In [1]:
import torch
import numpy as np
import os
import json


from torch.utils.data import DataLoader, ConcatDataset
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import svm


In [4]:
import sys

sys.path.append("../../")

import meerkats
import config

from meerkats import src
from meerkats.src.models.Palazcnn import PalazCNN
from meerkats.src.models.lit import Lit
from meerkats.src.data.nccrmeerkatsdataset import NCCRMeerkatsDataset
from meerkats.src.data.isabelmeerkatdataset import isabelMerkatDataset
from meerkats.src.utils import utils
from meerkats.src.features_extraction.extract_feats_segments import smile_features


/idiap/home/ibmahmoud/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
MAIN_DIR=config.GITROOT + "/experiments/" #dir for checkpoints
DATA_DIR=config.DATAMARA   # to change depend on data

Extract last layer of best cnn model

In [6]:

# lit=Lit(model=PalazCNN(n_input=1,n_output=6),num_classes=6,learning_rate=0.003)

# checkpoint=torch.load(MAIN_DIR + "experiment_/checkpoints/epoch=99-step=9600-v1.ckpt",map_location=torch.device('cpu')) #best marta model
# lit.load_state_dict(checkpoint["state_dict"])
# lit.model.eval()
with open(config.GITROOT + "/meerkats/src/data/class_to_index_mara.json") as f:
        class_to_index = json.load(f)
data_test = isabelMerkatDataset(
            audio_dir=DATA_DIR,
            class_to_index=class_to_index,
            target_sample_rate=16000,
            train=False) 
   
    
data_train = isabelMerkatDataset(
        audio_dir=DATA_DIR,
        class_to_index=class_to_index,
        target_sample_rate=16000,
        train=True) 
result={}
k_folds=5
dataset=torch.utils.data.ConcatDataset([data_test,data_train])
labels=dataset.datasets[0].filelist.class_index.tolist()+ dataset.datasets[1].filelist.class_index.tolist()
kfold=StratifiedKFold(n_splits=k_folds,shuffle=True,random_state=42)

accuracies_folds=[]

for fold,(train_ids,test_ids) in enumerate(kfold.split(dataset,labels)):
        if fold==1:
                
                num_train = len(train_ids)
                split = int(np.floor(0.2* num_train))
                                        #numpy.random.shuffle(indices)

                                        # mask=dataset.filelist.index.isin(train_ids)
                                        # dataset_=dataset.filelist[mask] 
                                        # all_ids=numpy.arange(0,len(dataset))
                                        # mask=x.index.isin(train_ids)
                                        # dataset_=x[mask]
                                        # for k, (train_idx, valid_idx) in enumerate(kfold.split(dataset_,dataset_.class_index)):
                                        #         train_ids=train_idx
                                        #         break
                                        # all_ids=numpy.arange(0,len(dataset))
                                        # keep_mask=numpy.isin(all_ids,test_ids,invert=True)
                                        # new_ids=all_ids[keep_mask]
                                        # train_ids=new_ids[train_ids]
                                        # valid_idx=new_ids[valid_idx]
                print(f'There are {len(test_ids)} data points in the test set ')
                print(f'There are {len(train_ids)} data points in the train set') 
                train_subsampler=torch.utils.data.SubsetRandomSampler(train_ids)
                test_subsampler=torch.utils.data.SubsetRandomSampler(test_ids)

                                # Dataloader in this fold
                train_loader = DataLoader(dataset, batch_size=1,sampler=train_subsampler,collate_fn=utils.collate_fn, num_workers=4)
                test_loader = DataLoader(dataset, batch_size=1,sampler=test_subsampler, shuffle=False,collate_fn=utils.collate_fn, num_workers=4)

              

There are 1286 data points in the test set 
There are 5142 data points in the train set


/idiap/home/ibmahmoud/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [79]:
  FEATS=[] # list of all the features
                for batch in iter(test_loader):
                        input,labels=batch
                # forward pass [with feature extraction]
                        x,intermediate=lit.model(input)
                        output=np.c_[intermediate.detach().numpy(),labels.detach().numpy()]
                        FEATS.append(output)
                        
                FEATS_test=np.array(FEATS)
                FEATS_test=FEATS_test.reshape((FEATS_test.shape[0]*FEATS_test.shape[1]), FEATS_test.shape[2]) # final features numpy
                FEATS=[]
                for batch in iter(train_loader):
                        input,labels=batch
                # forward pass [with feature extraction]
                        x,intermediate=lit.model(input)
                        output=np.c_[intermediate.detach().numpy(),labels.detach().numpy()]
                        FEATS.append(output)
                        
                FEATS_train=np.array(FEATS)
                print(FEATS_train[:,-1])
                FEATS_train=FEATS_train.reshape((FEATS_train.shape[0]*FEATS_train.shape[1]), FEATS_train.shape[2]) # final features numpy



IndentationError: unexpected indent (29062388.py, line 2)

In [7]:
filelist=pd.concat((dataset.datasets[0].filelist,dataset.datasets[1].filelist))

mask=filelist.index.isin(train_ids)
dataset_train=filelist[mask] 
mask=filelist.index.isin(test_ids)
dataset_test=filelist[mask]
                                        

In [9]:
smile=smile_features(dataset_train.path)
print(callable(smile))
smile.extract()

False


AttributeError: name

In [ ]:
param_grid = {
    "n_estimators": [0,10,20,30,40,50, 70,80, 100, 150, 200],
    "max_depth": [None, 5, 7 , 10],
    "min_samples_split": [2, 3,5, 7,10],
    "min_samples_leaf": [1, 2, 3,4]
}


In [ ]:
features_test=FEATS_test[:,:-1]
labels_test=FEATS_test[:,-1]
features_train=FEATS_train[:,:-1]
labels_train=FEATS_train[:,-1]


In [ ]:
print(FEATS_test.shape)
print(labels_train)

RANDOM FOREST

In [ ]:
rf=RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(features_train, labels_train)


In [ ]:
best_model=grid_search.best_estimator_
y_pred=best_model.predict(features_test)

In [ ]:
y_pred


In [ ]:
labels_test

In [ ]:
confusion=metrics.confusion_matrix(labels_test,y_pred)
confusion

In [ ]:
np.sum(np.diag(confusion) / np.sum(confusion,axis=1))/6

SVM TEST

In [ ]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['linear','rbf', 'poly', 'sigmoid']}
grid=GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
grid.fit(features_train,labels_train)

In [ ]:
grid_predictions = grid.predict(features_test)
confusion_m=metrics.confusion_matrix(labels_test,grid_predictions)
confusion_m

In [ ]:
np.sum(np.diag(confusion_m) / np.sum(confusion_m,axis=1))/6